In [1]:
import sys
sys.path.append("..")
import torch
import torch.nn.functional as F
import torchaudio
from omegaconf import OmegaConf
from ccreaim.model import operate
from ccreaim.utils import dataset, cfg_classes, audio_tools, util

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
## Maestro
data_tar = "/scratch/other/sopi/CCREAIM/datasets/maestro_bank_training_aug.tar"
data_tar_non_aug = "/scratch/other/sopi/CCREAIM/datasets/maestro_bank_training.tar"
samples_tar = "/scratch/other/sopi/CCREAIM/datasets/maestro_bank_samples.tar"

#load_trf_path = "/scratch/other/sopi/CCREAIM/logs/2023-03-10/bank-classifier_train_14-28-06/0/checkpoints/bank-classifier_seqlen-8_bs-32_lr-0.0001_seed-0_ep-1000.pt"
#load_trf_path = "/scratch/other/socapi/CCREAIM/logs/2023-03-15/bank-classifier_train_13-06-32/0/checkpoints/bank-classifier_seqlen-8_bs-4_lr-0.0001_seed-0_ep-030.pt"
#load_trf_path = "/scratch/other/sopi/CCREAIM/logs/2023-03-15/bank-classifier_train_15-26-42/0/checkpoints/bank-classifier_seqlen-8_bs-4_lr-0.0001_seed-0_ep-060.pt"
#load_trf_path = "/scratch/other/sopi/CCREAIM/logs/2023-03-15/bank-classifier_train_15-26-42/1/checkpoints/bank-classifier_seqlen-8_bs-4_lr-0.0001_seed-0_final.pt"
load_trf_path = "/scratch/other/sopi/CCREAIM/logs/2023-04-27/bank-classifier_train_08-53-33/0/checkpoints/bank-classifier_seqlen-8_bs-4_lr-0.0001_seed-0_final.pt"
context_length = 8
sample_rate = 44100

In [3]:
## Sounds
data_tar = "/scratch/other/sopi/CCREAIM/datasets/samples_sound_bank_training_aug.tar"
data_tar_non_aug = "/scratch/other/sopi/CCREAIM/datasets/samples_sound_bank_training.tar"
samples_tar = "/scratch/other/sopi/CCREAIM/datasets/sounds_bank_samples.tar"

# With pitch shift
load_trf_path = "/scratch/other/sopi/CCREAIM/logs/2023-04-27/bank-classifier_train_00-56-55/0/checkpoints/bank-classifier_seqlen-8_bs-4_lr-0.0001_seed-0_final.pt"

context_length = 8
sample_rate = 44100

In [4]:
# Load transformer 

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

trf_checkpoint = torch.load(load_trf_path, map_location="cpu")
trf_state_dict = trf_checkpoint["model_state_dict"]
hyper_cfg_schema = OmegaConf.structured(cfg_classes.HyperConfig)
trf_conf = OmegaConf.create(trf_checkpoint["hyper_config"])
del trf_conf["num_seq"]
del trf_conf["seq_cat"]
del trf_conf["pre_trained_ae_path"]
del trf_conf["pre_trained_vqvae_path"]
trf_hyper_cfg = OmegaConf.merge(hyper_cfg_schema, trf_conf)
trf_hyper_cfg.transformer.dropout = 0.0 # Since we're using trf.train() for no cacheing, set this to avoid dropout in inference
get_trf = operate.get_model_init_function(trf_hyper_cfg)
trf = get_trf()
trf.load_state_dict(trf_state_dict)
trf = trf.to(device)
trf

CachedDecoderOnly(
  (positional_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (transformer_decoder): CachedTransformerEncoder(
    (layers): ModuleList(
      (0): CachedTransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=612, out_features=612, bias=True)
        )
        (linear1): Linear(in_features=612, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=612, bias=True)
        (norm1): LayerNorm((612,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((612,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (trf_out_to_tokens): Linear(in_features=612, out_features=8192, bias=True)
)

In [5]:
# Prepare datasets
tmp_data_root_aug = dataset.prepare_dataset_on_tmp(data_tar)
data_aug = dataset.BankTransformerDataset(tmp_data_root_aug)
tmp_data_root_non_aug = dataset.prepare_dataset_on_tmp(data_tar_non_aug)
data_non_aug = dataset.BankTransformerDataset(tmp_data_root_non_aug)
# bank = dataset.Bank(tmp_data_root, context_length)

/tmp/samples_sound_bank_training_aug 11264
/tmp/samples_sound_bank_training 1024


In [6]:
tmp_data_root_samples = dataset.prepare_dataset_on_tmp(samples_tar)
data_samples = dataset.AudioDataset(tmp_data_root_samples, sample_rate)
print(len(data_samples))

/tmp/sounds_bank_samples 8192
8192


In [7]:
# list of indices with "mistakes" = 100, 250, 300, 350, 450
features, real_indices, _ = data_aug[100]

In [8]:
with torch.inference_mode():
    features_ = features.unsqueeze(0)
    features_ = features_.to(device)
    tgt = torch.cat(
            (
                torch.zeros_like(features_[:, 0:1, :], device=features_.device),
                features_[:, :-1, :],
            ),
            dim=1,
        )
    print(tgt.size())

    tgt_mask = util.get_tgt_mask(tgt.size(1))
    tgt_mask = tgt_mask.to(tgt.device)

    trf = trf.train()

    pred = trf(tgt, tgt_mask=tgt_mask)
    indices = pred.argmax(-1)
    print(indices)
    print(real_indices.long())
    probabilities = F.softmax(pred, dim=-1)
    print(probabilities.max(dim=2))
    print(probabilities[:,:,4075])

torch.Size([1, 8, 612])
tensor([[2488,   73,  457,   75,   75,   77,   78,   79]])
tensor([72, 73, 74, 75, 76, 77, 78, 79])
torch.return_types.max(
values=tensor([[0.0020, 1.0000, 0.1765, 1.0000, 0.3837, 1.0000, 1.0000, 1.0000]]),
indices=tensor([[2488,   73,  457,   75,   75,   77,   78,   79]]))
tensor([[1.1025e-06, 1.1428e-19, 2.5855e-10, 4.7627e-14, 1.8113e-08, 1.1227e-18,
         7.9664e-22, 1.9558e-19]])


In [9]:
def play_prediction(dataset, indices, sample_rate, sample=None):
    print(indices)
    for ind in indices:
        if sample == None:
            sample, name = dataset[ind.item()]
        else:
            next_sample, name = dataset[ind.item()]
            sample = torch.cat((sample, next_sample),dim=1)
        print(name)
    audio_tools.play_audio(sample, sample_rate)
        
    
play_prediction(data_samples, indices.int().squeeze()[1:], sample_rate)

tensor([ 73, 457,  75,  75,  77,  78,  79], dtype=torch.int32)
/tmp/sounds_bank_samples/ind_00073_22_context_0_sample1.wav
/tmp/sounds_bank_samples/ind_00457_29-11-2009_context_3_sample1.wav
/tmp/sounds_bank_samples/ind_00075_22_context_0_sample3.wav
/tmp/sounds_bank_samples/ind_00075_22_context_0_sample3.wav
/tmp/sounds_bank_samples/ind_00077_22_context_0_sample5.wav
/tmp/sounds_bank_samples/ind_00078_22_context_0_sample6.wav
/tmp/sounds_bank_samples/ind_00079_22_context_0_sample7.wav


In [10]:
def save_prediction(path, dataset, indices, sample_rate, sample=None):
    print(indices)
    for ind in indices:
        if sample == None:
            sample, name = dataset[ind.item()]
        else:
            next_sample, name = dataset[ind.item()]
            sample = torch.cat((sample, next_sample),dim=1)
        print(name)
    torchaudio.save(path, sample, sample_rate)

In [11]:
# THIS DOESN'T WORK WITH THE AUGMENTED DATASET
sample_id = 450
features1, real_indices1, _ = data_aug[sample_id]
features2, real_indices2, _ = data_aug[sample_id+1]

In [12]:
# Transformer has seen 
# [80, 81, 82, 83, 84, 85, 86, 87]
# and 
# [88, 89, 90, 91, 92, 93, 94, 95]
# But let's show it 
# [84, 85, 86, 87, 88, 89, 90, 91]

# Result: The transformer can't even predict 88 from 85,86,87
# so it hasn't really even learned anything :D 
with torch.inference_mode():
    features3 = torch.cat(
                  (
                      features1[context_length//2:context_length, :],
                      features2[0:context_length//2, :],
                  ),
                  dim=0,
        )
    print(features3.size())
    features3_ = features3.unsqueeze(0)
    features3_ = features3_.to(device)
    tgt = torch.cat(
            (
                torch.zeros_like(features3_[:, 0:1, :], device=features3_.device),
                features3_[:, :-1, :],
            ),
            dim=1,
        )
    print(tgt.size())

    tgt_mask = util.get_tgt_mask(tgt.size(1))
    tgt_mask = tgt_mask.to(tgt.device)

    trf = trf.train()
    pred = trf(tgt, tgt_mask=tgt_mask)
    print(pred.shape)
    indices = pred.argmax(-1)
    print(real_indices1)
    print(real_indices2)
    print(indices)

torch.Size([8, 612])
torch.Size([1, 8, 612])
torch.Size([1, 8, 8192])
tensor([320., 321., 322., 323., 324., 325., 326., 327.])
tensor([328., 329., 330., 331., 332., 333., 334., 335.])
tensor([[2488,  325,  326,  371,  300,  329,  330,  331]])


In [13]:
# This hacky function works only with the non-augmented dataset, can be used for POC generation
def get_features_from_index(ind, dataset, context_length):
    context, indices, a = dataset[ind//context_length]
    assert ind == indices[ind % context_length]
    return context[ind % context_length]
    

In [14]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k >0: keep only top k tokens with highest probability (top-k filtering).
            top_p >0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

In [15]:
def create_feature_vec_from_clip(x, Fs, frame_size, frame_step, deltas):
    F, f_names = ShortTermFeatures.feature_extraction(
        x, Fs, frame_size * Fs, frame_step * Fs, deltas=deltas
    )
    features = torch.tensor(F).view(-1)
    return features

In [16]:
# Returns both a scipy.io wavfile output and torchaudio.load output
def return_clip_from_path(path, starting_point_sec, clip_length_sec):
    torch_data, samp_rate = torchaudio.load(path)
    [Fs, x] = audioBasicIO.read_audio_file(path)
    
    if len(x.shape) != 1:
        x = numpy.average(x, axis=1).astype(x.dtype)
    if len(torch_data.shape) != 1:
        torch_data = torch.mean(torch_data, dim=0)

    torch_clip = torch_data[samp_rate*starting_point_sec : samp_rate*(starting_point_sec + clip_length_sec)]
    clip = x[samp_rate*starting_point_sec : samp_rate*(starting_point_sec + clip_length_sec)]
    return clip, torch_clip.unsqueeze(0), samp_rate

In [17]:
from pyAudioAnalysis import ShortTermFeatures, audioBasicIOdataset
import numpy
frame_size = 0.2
frame_step = 0.1
deltas = True
latent_dim = 612 # number of features returned per 1 second clip by ShortTermFeatures

# external_sample_path = "/scratch/other/sopi/CCREAIM/datasets/maestro-v3.0.0/2018/MIDI-Unprocessed_Schubert7-9_MID--AUDIO_11_R2_2018_wav.wav"
# external_sample_path = "/scratch/other/sopi/CCREAIM/datasets/maestro-v3.0.0/2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_17_R1_2018_wav--2.wav"
# external_sample_path = "/scratch/other/sopi/CCREAIM/datasets/maestro-v3.0.0/2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_01_R1_2018_wav--1.wav"
# external_sample_path = "/scratch/other/sopi/CCREAIM/datasets/maestro-v3.0.0/2017/MIDI-Unprocessed_083_PIANO083_MID--AUDIO-split_07-09-17_Piano-e_2_-06_wav--4.wav"
# external_sample_path = "/scratch/other/sopi/CCREAIM/datasets/maestro-v3.0.0/2018/MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1.wav"
# external_sample_path = "/scratch/other/sopi/CCREAIM/datasets/samples/99-12-2009.wav"

# test_fn = "ethereal-teleport"
# test_fn = "across"
# test_fn = "quartet"
# test_fn = "piano_man"
# test_fn = "train-upon-us"
test_fn = "waves-of-hawaii"
external_sample_path = "/scratch/other/sopi/CCREAIM/datasets/test/in/{}.wav".format(test_fn)

# Generation
top_p = 0
top_k = 5
temperature = 1.0

external = True
start_sec = 0

sample_id = 1385

n = context_length // 2
torch_sample = None
with torch.inference_mode():
    # Pick half of context length, generate rest
    if external:
        features_ = torch.zeros(1, n, latent_dim)
        indices = torch.zeros(n)
        for i in range(n):
            clip, torch_clip, samp_rate = return_clip_from_path(external_sample_path, start_sec+i, 1)
            print(clip.shape, torch_clip.shape, samp_rate)
            feature_vec = create_feature_vec_from_clip(clip, samp_rate, frame_size, frame_step, deltas)
            print(feature_vec.shape)
            features_[:,i,:] = feature_vec
            if torch_sample == None:
                torch_sample = torch_clip
            else:
                torch_sample = torch.cat((torch_sample, torch_clip),dim=1)
    else:
        features, indices, _ = data_aug[sample_id]
        features_ = features.unsqueeze(0)
    features_ = features_.to(device)
    
    features_half = features_[:, :n, :]
    tgt = torch.cat(
            (
                torch.zeros_like(features_half[:, 0:1, :], device=features_.device),
                features_half[:, :-1, :],
            ),
            dim=1,
        )
    print(tgt.size())
    
    # No cacheing
    trf = trf.train()
    indices = indices[0:(n-1)]
    print(indices)
    for i in range(n):
        print(tgt.shape)
        trf_out = trf(tgt)
        print("hello", trf_out.shape)
        trf_out_filtered = top_k_top_p_filtering(trf_out[:,-1,:].squeeze()/temperature, top_k=top_k, top_p=top_p)
        probabilities = F.softmax(trf_out_filtered)
        print("max_prob:", max(probabilities))
        emb_ind = torch.multinomial(probabilities,1)
        print(emb_ind)
        # print("optional_emb_ind:", trf_out[:,-1,:].argmax(-1))
        #emb_ind = trf_out[:,-1,:].argmax(-1)
        indices = torch.cat((indices, emb_ind), dim=0)
        next_feature = get_features_from_index(emb_ind.item(), data_non_aug, context_length)
        print("features:", next_feature.shape)
        tgt = torch.cat(
                (
                    tgt,
                    next_feature.unsqueeze(0).unsqueeze(0)
                ),
                dim=1
            )
        probabilities = F.softmax(trf_out, dim=-1)
        print(probabilities.max(dim=2)[0][-1][-1])
        print(probabilities.max(dim=2)[1][-1][-1])
    print(indices)

/scratch/other/sopi/conda/ccreaim/lib/python3.9/site-packages/pyAudioAnalysis-0.3.14-py3.9.egg/pyAudioAnalysis/audioBasicIO.py:99: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


(48000,) torch.Size([1, 48000]) 48000
torch.Size([612])
(48000,) torch.Size([1, 48000]) 48000
torch.Size([612])
(48000,) torch.Size([1, 48000]) 48000
torch.Size([612])
(48000,) torch.Size([1, 48000]) 48000
torch.Size([612])
torch.Size([1, 4, 612])
tensor([0., 0., 0.])
torch.Size([1, 4, 612])
hello torch.Size([1, 4, 8192])


/tmp/ipykernel_22278/1001478765.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probabilities = F.softmax(trf_out_filtered)


max_prob: tensor(0.4084)
tensor([4523])
features: torch.Size([612])
tensor(0.3010)
tensor(4523)
torch.Size([1, 5, 612])
hello torch.Size([1, 5, 8192])
max_prob: tensor(1.)
tensor([4524])
features: torch.Size([612])
tensor(1.)
tensor(4524)
torch.Size([1, 6, 612])
hello torch.Size([1, 6, 8192])
max_prob: tensor(1.0000)
tensor([4525])
features: torch.Size([612])
tensor(1.0000)
tensor(4525)
torch.Size([1, 7, 612])
hello torch.Size([1, 7, 8192])
max_prob: tensor(1.)
tensor([4526])
features: torch.Size([612])
tensor(1.)
tensor(4526)
tensor([   0.,    0.,    0., 4523., 4524., 4525., 4526.])


In [18]:
play_prediction(data_samples, indices[3:].int(), sample_rate, torch_sample)

tensor([4523, 4524, 4525, 4526], dtype=torch.int32)
/tmp/sounds_bank_samples/ind_04523_recording 2021-11-22 13-59-28_context_83_sample3.wav
/tmp/sounds_bank_samples/ind_04524_recording 2021-11-22 13-59-28_context_83_sample4.wav
/tmp/sounds_bank_samples/ind_04525_recording 2021-11-22 13-59-28_context_83_sample5.wav
/tmp/sounds_bank_samples/ind_04526_recording 2021-11-22 13-59-28_context_83_sample6.wav


In [19]:
import torchaudio
save = True
#save = False
save_path_root = "bank_outputs/maestro"
if save:
    save_prediction("{}/{}/{}-gen3-sec{}.wav".format(save_path_root, test_fn, test_fn, start_sec), data_samples, indices[3:].int(), sample_rate, torch_sample)

tensor([4523, 4524, 4525, 4526], dtype=torch.int32)
/tmp/sounds_bank_samples/ind_04523_recording 2021-11-22 13-59-28_context_83_sample3.wav
/tmp/sounds_bank_samples/ind_04524_recording 2021-11-22 13-59-28_context_83_sample4.wav
/tmp/sounds_bank_samples/ind_04525_recording 2021-11-22 13-59-28_context_83_sample5.wav
/tmp/sounds_bank_samples/ind_04526_recording 2021-11-22 13-59-28_context_83_sample6.wav


RuntimeError: Error saving audio file: failed to open file bank_outputs/maestro/waves-of-hawaii/waves-of-hawaii-gen3-sec0.wav